In [1]:
import numpy as np
import pandas as pd

In [19]:
data_train = pd.read_csv('yelp_academic_dataset_review_train.csv')
data_test = pd.read_csv('yelp_academic_dataset_review_test.csv')

In [22]:
all_reviews = pd.concat([data_train.text, data_test.text])
train_length = data_train.shape[0]
test_length = data_test.shape[0]

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_predict

In [40]:
count_vect = CountVectorizer()
X_counts = count_vect.fit_transform(all_reviews)
X_train_counts = X_counts[:train_length]
X_test_counts = X_counts[train_length:]
tfidf_transformer = TfidfTransformer()
X_tfidf = tfidf_transformer.fit_transform(X_counts) # For some reason this doesn't work
X_train_tfidf = X_tfidf[:train_length] # For some reason this doesn't work
X_test_tfidf = X_tfidf[train_length:] # For some reason this doesn't work

In [44]:
clf = MultinomialNB()
clf.fit(X_train_counts, data_train.stars) # Not using tf-idf, just bag of words
preds = clf.predict(X_test_counts)
np.mean(preds)

3.6525845157169581

In [45]:
data_test["preds"] = preds
pred_per_buisiness = data_test.groupby("business_id").mean().preds
test_biz = pd.read_csv("yelp_academic_dataset_business_test.csv")

In [95]:
pred_df = pd.DataFrame([{"business_id": biz, "stars": pred_per_buisiness[biz]} for biz in test_biz.business_id])
pred_df = pred_df.set_index("business_id")
pred_df.to_csv("Submission")